In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# IAM Entitlement Analysis Notebook\n",
    "## CloudGuardStack - Interactive IAM Security Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import json\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from IPython.display import display, Markdown\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "\n",
    "# Set up plotting style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Entitlement Report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the entitlement report\n",
    "with open('../entitlement_report.json', 'r') as f:\n",
    "    report = json.load(f)\n",
    "\n",
    "display(Markdown(f\"### Scan Overview\"))\n",
    "display(Markdown(f\"- **Scan Time**: {report['scan_metadata']['scan_time']}\"))\n",
    "display(Markdown(f\"- **Account ID**: {report['scan_metadata']['account_id']}\"))\n",
    "display(Markdown(f\"- **Entities Scanned**: {report['scan_metadata']['entities_scanned']}\"))\n",
    "display(Markdown(f\"- **Total Findings**: {report['scan_metadata']['total_findings']}\"))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Risk Score Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create risk score DataFrame\n",
    "risk_df = pd.DataFrame.from_dict(report['risk_scores'], orient='index', columns=['risk_score'])\n",
    "risk_df = risk_df.sort_values('risk_score', ascending=False)\n",
    "\n",
    "print(\"Top 10 Riskiest Entities:\")\n",
    "display(risk_df.head(10))\n",
    "\n",
    "# Plot risk distribution\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Risk score distribution\n",
    "ax1.hist(risk_df['risk_score'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "ax1.set_title('Distribution of Risk Scores')\n",
    "ax1.set_xlabel('Risk Score')\n",
    "ax1.set_ylabel('Number of Entities')\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Top entities by risk\n",
    "top_10 = risk_df.head(10)\n",
    "ax2.barh(top_10.index, top_10['risk_score'], color='coral', alpha=0.7)\n",
    "ax2.set_title('Top 10 Riskiest Entities')\n",
    "ax2.set_xlabel('Risk Score')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Finding Type Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze finding types\n",
    "findings_data = []\n",
    "for finding_type, items in report['findings'].items():\n",
    "    for item in items:\n",
    "        findings_data.append({\n",
    "            'finding_type': finding_type,\n",
    "            'entity_name': item.get('entity_name', 'unknown'),\n",
    "            'risk_level': item.get('risk_level', 'MEDIUM')\n",
    "        })\n",
    "\n",
    "findings_df = pd.DataFrame(findings_data)\n",
    "\n",
    "# Finding type counts\n",
    "finding_counts = findings_df['finding_type'].value_counts()\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "# Finding types pie chart\n",
    "ax1.pie(finding_counts.values, labels=finding_counts.index, autopct='%1.1f%%', startangle=90)\n",
    "ax1.set_title('Finding Types Distribution')\n",
    "\n",
    "# Risk level by finding type\n",
    "risk_by_type = findings_df.groupby(['finding_type', 'risk_level']).size().unstack(fill_value=0)\n",
    "risk_by_type.plot(kind='bar', ax=ax2, stacked=True)\n",
    "ax2.set_title('Risk Levels by Finding Type')\n",
    "ax2.set_xlabel('Finding Type')\n",
    "ax2.set_ylabel('Count')\n",
    "ax2.legend(title='Risk Level')\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Interactive Visualization with Plotly"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create interactive risk score visualization\n",
    "fig = go.Figure()\n",
    "\n",
    "# Add risk scores as bars\n",
    "fig.add_trace(go.Bar(\n",
    "    x=risk_df.index[:15],  # Top 15 entities\n",
    "    y=risk_df['risk_score'][:15],\n",
    "    marker_color=['#FF6B6B' if score >= 80 else '#FFA726' if score >= 60 else '#FFE082' for score in risk_df['risk_score'][:15]]\n",
    "))\n",
    "\n",
    "fig.update_layout(\n",
    "    title='Top 15 IAM Entities by Risk Score',\n",
    "    xaxis_title='IAM Entities',\n",
    "    yaxis_title='Risk Score',\n",
    "    showlegend=False,\n",
    "    height=500\n",
    ")\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Remediation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze remediation suggestions\n",
    "remediation_df = pd.DataFrame(report['remediation_suggestions'])\n",
    "\n",
    "if not remediation_df.empty:\n",
    "    print(\"Remediation Priority Summary:\")\n",
    "    priority_summary = remediation_df['priority'].value_counts()\n",
    "    \n",
    "    fig = px.pie(\n",
    "        values=priority_summary.values,\n",
    "        names=priority_summary.index,\n",
    "        title='Remediation Priority Distribution',\n",
    "        color=priority_summary.index,\n",
    "        color_discrete_map={\n",
    "            'CRITICAL': '#FF6B6B',\n",
    "            'HIGH': '#FFA726',\n",
    "            'MEDIUM': '#FFE082',\n",
    "            'LOW': '#C8E6C9'\n",
    "        }\n",
    "    )\n",
    "    \n",
    "    fig.show()\n",
    "    \n",
    "    # Display high priority remediations\n",
    "    high_priority = remediation_df[remediation_df['priority'].isin(['CRITICAL', 'HIGH'])]\n",
    "    if not high_priority.empty:\n",
    "        display(Markdown(\"### High Priority Remediation Actions\"))\n",
    "        for _, row in high_priority.iterrows():\n",
    "            display(Markdown(f\"**{row['entity']}** - {row['suggestion']}\"))\n",
    "            display(Markdown(f\"*Steps:* {', '.join(row['steps'])}\"))\n",
    "            display(Markdown(\"---\"))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Export Analysis Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate summary report\n",
    "summary_data = {\n",
    "    'total_entities': report['scan_metadata']['entities_scanned'],\n",
    "    'total_findings': report['scan_metadata']['total_findings'],\n",
    "    'critical_findings': report['summary']['critical_findings'],\n",
    "    'high_findings': report['summary']['high_findings'],\n",
    "    'medium_findings': report['summary']['medium_findings'],\n",
    "    'low_findings': report['summary']['low_findings'],\n",
    "    'average_risk_score': risk_df['risk_score'].mean(),\n",
    "    'max_risk_score': risk_df['risk_score'].max()\n",
    "}\n",
    "\n",
    "summary_df = pd.DataFrame([summary_data])\n",
    "display(Markdown(\"### Executive Summary\"))\n",
    "display(summary_df.T.rename(columns={0: 'Metrics'}))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}